In [4]:
!pip install nltk
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [5]:
nltk.download('punkt')
nltk.download('stopwords')
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
from google.colab import files
pd= files.upload()

Saving IMDB Dataset.csv to IMDB Dataset.csv


In [147]:
import pandas as pd
reviews_df = pd.read_csv('IMDB Dataset.csv')

In [148]:
def count_tokens(text):
    return len(text.split())

reviews_df['token_count'] = reviews_df['review'].apply(count_tokens)

df_filtered = reviews_df[reviews_df['token_count'] <= 128]


In [149]:
df_2000 = df_filtered.sample(n=2000, random_state=1)

In [150]:
def preprocess_review(review):
    review = review.lower()
    review = re.sub(r'[^a-z\s]', '', review)
    words = word_tokenize(review)
    cleaned_review = [stemmer.stem(word) for word in words if word not in stop_words]
    return ' '.join(cleaned_review)


In [151]:
df_2000['cleaned_review'] = df_2000['review'].apply(preprocess_review)

In [152]:
df_2000_final = df_2000.drop(columns=['review', 'token_count'])

In [153]:
df_2000_final.to_csv('processed_reviews_128.csv', index=False)
files.download('processed_reviews_128.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [154]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

In [155]:
pd= files.upload()

Saving processed_reviews_128 (6).csv to processed_reviews_128 (6).csv


In [107]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

In [157]:
import pandas as pd
reviews_df = pd.read_csv('processed_reviews_128 (6).csv')

In [158]:
reviews_df

,sentiment,cleaned_review
0,positive,mani funni scene peopl dont normal pay attent ...
1,negative,shame shame fine actor joseph fienn would allo...
2,negative,movi pretti predict nuff saidfrom delay kiss s...
3,positive,black mask jet li play bioengin superkil turn ...
4,negative,watch entir movi recogn particip william hurt ...
...,...,...
1995,negative,horribl mish mash predict stori line toebendin...
1996,negative,thoughtless ignor illconceiv careerkil talent ...
1997,positive,saw film ae channel past weekend mysteri okay ...
1998,negative,home weekend bore lack move absolut noth bette...


In [159]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(reviews_df['cleaned_review'])

In [160]:
positive_indices = reviews_df[reviews_df['sentiment'] == 'positive'].index
negative_indices = reviews_df[reviews_df['sentiment'] == 'negative'].index

In [161]:
negative_indices

Int64Index([   1,    2,    4,    5,    7,    8,    9,   11,   12,   16,
            ...
            1988, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1998, 1999],
           dtype='int64', length=989)

In [162]:
num_clusters = 2
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
clusters = kmeans.fit_predict(X)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [163]:
def find_closest_to_centroid(indices, cluster_label, cluster_centers, X):
    filtered_points = X[indices]
    closest, _ = pairwise_distances_argmin_min(cluster_centers, filtered_points)
    return indices[closest[cluster_label]]

In [164]:
average_positive_cluster_label = clusters[positive_indices].mean().astype(int)
average_negative_cluster_label = clusters[negative_indices].mean().astype(int)

In [165]:
closest_positive_review_index = find_closest_to_centroid(positive_indices, average_positive_cluster_label, kmeans.cluster_centers_, X)
closest_negative_review_index = find_closest_to_centroid(negative_indices, average_negative_cluster_label, kmeans.cluster_centers_, X)

In [166]:
closest_positive_review = reviews_df.loc[closest_positive_review_index]['cleaned_review']
closest_negative_review = reviews_df.loc[closest_negative_review_index]['cleaned_review']

# Print the closest reviews
print("Closest positive review:\n", closest_positive_review)
print("\nClosest negative review:\n", closest_negative_review)


Closest positive review:
 peopl call comedi watch laughedbr br guess problem first saw br br wasnt old enough understand wasnt meant bebr br taken serious quit scene meantbr br funni care much charact laugh atbr br thembr br suggest watch film next time your fall lovebr br tri take serious think youll find despit sillybr br flaw one move love stori youv ever seen

Closest negative review:
 read review decid rent dvd version br br like classic music want learn bach br br disappoint guess know enough bach music comment enough understand import music play br br mayb would appropri guidanc expert bach music explain film br br realli tri saw whole film hop would abl enjoy least br br see risk
